In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from action import Action
# from PDDL import PDDL_Parser
from PDDLz3 import PDDL_Parser_z3
import sys, pprint
from collections import OrderedDict
from typing import List, Tuple, Dict, Iterable
import re, copy, json
import itertools
import z3
from skill_classes import EffectTypePDDL, SkillPDDL
from utils import product_dict, nested_list_replace, get_atoms, get_all_objects, condition_str2objects
from scoping import scope
import time

In [9]:
def pvars2objects(pvars):
    objs = condition_str2objects(map(str,pvars))
    objs = [s.strip() for s in objs]
    objs = sorted(list(set(objs)))
    return objs

In [26]:
def scope_file(domain, problem):
    parser = PDDL_Parser_z3()
    parser.parse_domain(domain)
    print("Parsed domain")
    parser.parse_problem(problem)
    print("Parsed problem")
    with open(problem, "r") as f:
        if ":metric" in f.read():
            raise ValueError(":metric makes the parser or scoper freeze")
    skill_list = parser.get_skills()
    print("Got skills")
    # This below block converts all the domain's goals to z3
    goal_cond = parser.get_goal_cond()
    print("Got goal")
    # This below block converts all the domain's initial conditions to z3
    init_cond_list = parser.get_init_cond_list()
    print("Got initial state")
    # Run the scoper on the constructed goal, skills and initial condition
    rel_pvars, rel_skills = scope(goals=goal_cond, skills=skill_list, start_condition=init_cond_list,verbose=0)

    all_pvars = []
    for s in skill_list:
        all_pvars.extend(get_atoms(s.precondition))
        all_pvars.extend(s.params)
    all_objects = pvars2objects(all_pvars)
    # print(all_objects)
    rel_objects = pvars2objects(rel_pvars)
    # print(rel_objects)
    irrel_objects = [x for x in all_objects if x not in rel_objects]
    irrel_pvars = [str(x) for x in all_pvars if str(x) not in map(str, rel_pvars)]
    print(irrel_objects)

    # return rel_objects, irrel_objects
    return rel_pvars, irrel_pvars


In [27]:
results = OrderedDict()

In [30]:
domprobs = [
    # ("examples/IPC_Domains/depot/DepotsNum.pddl", "examples/IPC_Domains/depot/prob-01.pddl"),
    # ("examples/IPC_Domains/zeno/zeno.pddl", "examples/IPC_Domains/zeno/pb1-bloated.pddl"),
            # ("examples/IPC_Domains/driverlog/driverlogNumeric.pddl", "examples/IPC_Domains/driverlog/pfile0"),
            # ("examples/IPC_Domains/freecell/free.pddl", "examples/IPC_Domains/freecell/pfile0"),
            ("examples/IPC_Domains/Rovers/NumRover.pddl", "examples/IPC_Domains/Rovers/prob-01.pddl"),
            # ("examples/IPC_Domains/Satellite/metricSat.pddl", "examples/IPC_Domains/Satellite/pfile0"),
            ]

for dom, prob in domprobs:
    print(f"({dom}, {prob})")
    start_time = time.time()
    try:
        rel, irrel = scope_file(dom, prob)
        # print(rel)
        r = irrel
    except Exception as e:
        r = e
    duration = time.time() - start_time
    r = (duration, r)
    print(r)
    results[(dom, prob)] = r
    

(examples/IPC_Domains/Rovers/NumRover.pddl, examples/IPC_Domains/Rovers/prob-01.pddl)
Parsed domain
Parsed problem
Got skills
Got goal
Got initial state
[]
(109.94428610801697, ['can_traverse(rover0, waypoint0, waypoint3)', 'can_traverse(rover0, waypoint1, waypoint2)', 'can_traverse(rover0, waypoint1, waypoint3)', 'can_traverse(rover0, waypoint2, waypoint1)', 'can_traverse(rover0, waypoint3, waypoint0)', 'can_traverse(rover0, waypoint3, waypoint1)', 'equipped_for_soil_analysis(rover0)', 'store_of(rover0store, rover0)', 'equipped_for_soil_analysis(rover0)', 'store_of(rover0store, rover0)', 'equipped_for_soil_analysis(rover0)', 'store_of(rover0store, rover0)', 'equipped_for_soil_analysis(rover0)', 'store_of(rover0store, rover0)', 'equipped_for_rock_analysis(rover0)', 'store_of(rover0store, rover0)', 'equipped_for_rock_analysis(rover0)', 'store_of(rover0store, rover0)', 'equipped_for_rock_analysis(rover0)', 'store_of(rover0store, rover0)', 'equipped_for_rock_analysis(rover0)', 'store_of(r

In [6]:
with open("./scope_results.json", "r") as f:
    json.dump(results, f, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: './scope_results.json'

In [21]:
for dom, prob in [("examples/Rovers/Strips/StripsRover.pddl", "examples/Rovers/Strips/pfile1")]:
    print(f"({dom}, {prob})")
    start_time = time.time()
    try:
        rel, irrel = scope_file(dom, prob)
        r = irrel
    except Exception as e:
        r = e
    duration = time.time() - start_time
    r = (duration, r)
    print(r)


(examples/Rovers/Strips/StripsRover.pddl, examples/Rovers/Strips/pfile1)
Parsed domain
Parsed problem
Got skills
Got goal
Got initial state
Splitting goals
Split goals
Exception ignored in: <function AstRef.__del__ at 0x7fbd7ac710e0>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/scoping/lib/python3.7/site-packages/z3/z3.py", line 313, in __del__
    if self.ctx.ref() is not None and self.ast is not None:
AttributeError: 'BoolSortRef' object has no attribute 'ctx'
(53.07485222816467, ArgumentError("argument 1: <class 'KeyboardInterrupt'>: "))
